In [1]:
import os
os.environ["R_HOME"] = "/home/ld/anaconda3/envs/r/lib/R"
import sys
sys.path.append('../')
import scDesign3

The R project used is located at /home/ld/anaconda3/envs/r/lib/R


In [2]:
import anndata as ad
example_sce = ad.read_h5ad('test_data/exampledata1.h5ad')
example_sce.obs['test'] = ['test' for i in range(example_sce.n_obs)]
# example_sce = example_sce[1:5,1:5]

In [3]:
example_sce.obs

,clusters_coarse,clusters,S_score,G2M_score,cell_type,sizeFactor,pseudotime,test
AAACCTGAGAGGGATA,Pre-endocrine,Pre-endocrine,-0.224902,-0.252071,Pre-endocrine,0.813018,0.677236,test
AAACCTGGTAAGTGGC,Ngn3 high EP,Ngn3 high EP,-0.179981,-0.126030,Ngn3 high EP,0.824834,0.431569,test
AAACGGGCAAAGAATC,Endocrine,Beta,-0.174345,-0.221602,Beta,1.045403,0.749373,test
AAACGGGGTACAGTTC,Endocrine,Beta,-0.186562,-0.157197,Beta,1.506890,0.925441,test
AAACGGGGTGAAATCA,Ngn3 high EP,Ngn3 high EP,-0.055381,1.020705,Ngn3 high EP,1.398903,0.380345,test
...,...,...,...,...,...,...,...,...
TTTGGTTTCACTTACT,Ngn3 high EP,Ngn3 high EP,0.079044,-0.300046,Ngn3 high EP,1.765861,0.103478,test
TTTGGTTTCCTTTCGG,Endocrine,Beta,-0.186055,-0.185440,Beta,0.628062,0.792967,test
TTTGTCAAGAATGTGT,Pre-endocrine,Pre-endocrine,-0.298142,-0.254821,Pre-endocrine,1.099888,0.699019,test
TTTGTCAAGTGACATA,Pre-endocrine,Pre-endocrine,-0.235896,-0.266101,Pre-endocrine,1.512962,0.590720,test


In [4]:
test = scDesign3.scDesign3()

construct_data interface

In [8]:
bpparam = test.get_bpparam(stop_on_error = False)
print(bpparam)

class: MulticoreParam
  bpisup: FALSE; bpnworkers: 4; bptasks: 0; bpjobname: BPJOB
  bplog: FALSE; bpthreshold: INFO; bpstopOnError: FALSE
  bpRNGseed: ; bptimeout: NA; bpprogressbar: FALSE
  bpexportglobals: TRUE; bpexportvariables: FALSE; bpforceGC: FALSE
  bpfallback: TRUE
  bplogdir: NA
  bpresultdir: NA
  cluster type: FORK



In [9]:
test.construct_data(
    anndata=example_sce,
    corr_by="1",
    default_assay_name="counts",
    celltype='cell_type',
    parallelization='bpmapply',
    bpparam=bpparam
)

In [10]:
print(test.construct_data_res)

$count_mat
                  Pyy Iapp Chgb Rbp4 Spp1 Chga Cck Ins1 Nnat Ins2 Neurog3
AAACCTGAGAGGGATA  103    0   86    8    0    4   6    0    0    0       1
AAACCTGGTAAGTGGC    1    0    2    0    0    7   2    0    2    0      12
AAACGGGCAAAGAATC    0    2   19  102    2   55   3    0    1    0       0
AAACGGGGTACAGTTC  256  508   23   54    0   29   2    1  182   96       0
AAACGGGGTGAAATCA    1    2    0    0    0    0  77    0    3    0       5
AAACGGGTCAAACAAG    1    1    0    0   10    0   1    1    2    0      28
AAACGGGTCAGCTCTC    1    0    0    0  105    0   1    0    0    0       1
AAACGGGTCCGCGCAA   31   19    0   16    1    1   5    0    0    0       0
AAACGGGTCGCATGGC   56   21    4   47    0    7   3    0   52    1       0
AAAGATGAGAGTACCG    0    0   15    0    0   58   1    0    0    0       0
AAAGATGAGTCATGCT   93   91   31  114    0   19   0   35  124   12       2
AAAGATGAGTCTTGCA  143  230   17   48    0   15   0   74   45   63       1
AAAGATGAGTGAACAT    0    0 

In [ ]:
# SummarizedExperiment = packages.importr('SummarizedExperiment')
# print(SummarizedExperiment.assay(test._sce,test._assay_use))
# print(SummarizedExperiment.colData(test._sce))
# print(SummarizedExperiment.rowData(test._sce))